In [ ]:
import numpy as np
from scipy.spatial import distance
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report

In [ ]:
seed = 0
np.random.seed(seed)

In [ ]:
train_file = 'data/Beef_TRAIN'

In [ ]:
test_file = 'data/Beef_TEST'

# Prep Train Data

In [ ]:
train_data = np.loadtxt(train_file)

In [ ]:
np.random.shuffle(train_data)

In [ ]:
X_train = train_data[:,1:]

In [ ]:
y_train = train_data[:,0]

In [ ]:
print(X_train[0])

In [ ]:
print(y_train)

# Prep Test Data

In [ ]:
test_data = np.loadtxt(test_file)

In [ ]:
np.random.shuffle(test_data)

In [ ]:
X_test = test_data[:,1:]

In [ ]:
y_test = test_data[:,0]

In [ ]:
print(X_test[0])

In [ ]:
print(y_test[0])

In [ ]:
print(type(X_test[0][1]))

# Train Model

In [ ]:
def DTW(a, b):
    # ถ้า DTW  ทำช้าอาจต้องทำ sampling เพื่อลดปริมาณ data ทำ DTW จะได้เร็วขึ้น    
    an = a.size
    bn = b.size
    pointwise_distance = distance.cdist(a.reshape(-1,1),b.reshape(-1,1))
    cumdist = np.matrix(np.ones((an+1,bn+1)) * np.inf)
    cumdist[0,0] = 0
    
    for ai in range(an):
        for bi in range(bn):
            minimum_cost = np.min([cumdist[ai, bi+1],
                                   cumdist[ai+1, bi],
                                   cumdist[ai, bi]])
            cumdist[ai+1, bi+1] = pointwise_distance[ai,bi] + minimum_cost

    return cumdist[an, bn]

In [ ]:
def DTWWithWeightNeighborCell(a,b,**kwargs):
    print("weight",alpha,beta,gamma)
    an = a.size
    bn = b.size
    pointwise_distance = distance.cdist(a.reshape(-1,1),b.reshape(-1,1))
    cumdist = np.matrix(np.ones((an+1,bn+1)) * np.inf)
    cumdist[0,0] = 0
    
    for ai in range(an):
        for bi in range(bn):
            minimum_cost = np.min([**kwargs["metric_params"]["alpha"]*cumdist[ai, bi+1],
                                   **kwargs["metric_params"]["beta"]*cumdist[ai+1, bi],
                                   **kwargs["metric_params"]["gamma"]*cumdist[ai, bi]])
            cumdist[ai+1, bi+1] = pointwise_distance[ai,bi] + minimum_cost

    return cumdist[an, bn]

In [ ]:
clf = KNeighborsClassifier(metric =DTW,n_neighbors=1,n_jobs=-1)
clf.fit(X_train[0:5], y_train[0:5])

# Test Model

In [ ]:
y_pred = clf.predict(X_test[0:5])

In [ ]:
print(classification_report(y_test[0:5], y_pred[0:5]))

# Create Model with weight neighboring cells

In [ ]:
import copy

In [ ]:
knnList = []
for i in range (1,3):
    for j in range (1,2):
        for k in range (1,2):
            knn = KNeighborsClassifier(metric =copy.deepcopy((lambda a,b:DTWWithWeightNeighborCell(a,b,i,j,k))),n_neighbors=1,n_jobs=-1)
            knn.fit(X_train[0:5], y_train[0:5])
            knnList.append(copy.deepcopy(knn))

In [ ]:
for knn in knnList:
    y_pred = knn.predict(X_test[0:5])
    print(classification_report(y_test[0:5], y_pred[0:5]))

In [ ]:
y_pred2 = knn2.predict(X_test[0:5])
print(classification_report(y_test[0:5], y_pred2[0:5]))

In [ ]:
y_pred3 = knn3.predict(X_test[0:5])
print(classification_report(y_test[0:5], y_pred3[0:5]))

In [ ]:
y_pred4 = knn4.predict(X_test[0:5])
print(classification_report(y_test[0:5], y_pred4[0:5]))